In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import csv
import cv2
import json
import uuid
import torch
import folium
import requests
import numpy as np
import transformers
import pandas as pd
import seaborn as sns
import torchvision.ops as ops
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from torchvision.ops import box_convert
from PIL import Image, ImageDraw, ImageFont
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
df_bm = pd.read_csv("/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/CSV Update/BeetleMeasurements_updated_merged_uniqueBeetles.csv")

In [3]:
df_bm["resized_image_filepath"] = df_bm["raw_filepath"].str.replace("group_images", "resized_images", regex=False)

In [4]:
df_bm.head(2)

,beetle_uuid,length_uuid,width_uuid,length_distance_pix,length_distance_cm,length_coord_value,length_coord_value_raw_image,width_distance_pix,width_distance_cm,width_coord_value,width_coord_value_raw_image,beetleID,pictureID,beetle_position,lying_flat,scalebar_coord,pix_per_cm,raw_filepath,raw_image_shape,resized_image_shape,scientificName,genus,species,user_name,workflowID,NEON_sampleID,siteID,collection_date,plotID,siteName,siteState,siteLatitude,siteLongitude,siteType,siteTerrain,evapotranspiration,max_temp,min_temp,precipitation,precipitation_hours,rain_sum,shortwave_radiation,siteDomainCode,snowfall_sum,sunshine_duration,weather_code,wind_gusts_max,wind_speed_max,resized_image_filepath
0,7e443d69-bdcb-4c50-bb56-a1d9142577a6_4e7238e8-aec2-4975-98ad-6e56cc5176c2,7e443d69-bdcb-4c50-bb56-a1d9142577a6,4e7238e8-aec2-4975-98ad-6e56cc5176c2,114.017543,1.117336,"{""x1"": 1047, ""y1"": 230, ""x2"": 1049, ""y2"": 344}","{'x1': 2991, 'y1': 657, 'x2': 2997, 'y2': 982}",65.030762,0.637281,"{""x1"": 1015, ""y1"": 247, ""x2"": 1080, ""y2"": 245}","{'x1': 2900, 'y1': 705, 'x2': 3086, 'y2': 699}",A00000001831_1,A00000001831.jpg,1,Yes,"{""x1"": 840, ""y1"": 343, ""x2"": 942, ""y2"": 340}",102.044108,group_images/A00000001831.jpg,"(3712, 5568, 3)","(1949, 1299, 3)",Chlaenius aestivus,Chlaenius,aestivus,IsaFluck,23248,SERC_010.20180523.CHLAES.01,SERC,2018-05-23,SERC_010,Smithsonian Environmental Research Center NEON,MD,38.89013,-76.56001,GRADIENT,TERRESTRIAL,4.27,26.6,18.9,2.0,6.0,2.0,22.67,D02,0.0,45603.39,61.0,34.6,14.6,resized_images/A00000001831.jpg
1,60aac143-086c-43a6-b506-0a366bba545d_db0890b5-5dfc-408b-b734-4df692166c84,60aac143-086c-43a6-b506-0a366bba545d,db0890b5-5dfc-408b-b734-4df692166c84,109.018347,1.068345,"{""x1"": 1055, ""y1"": 476, ""x2"": 1053, ""y2"": 585}","{'x1': 3014, 'y1': 1359, 'x2': 3009, 'y2': 1671}",64.000000,0.627180,"{""x1"": 1018, ""y1"": 490, ""x2"": 1082, ""y2"": 490}","{'x1': 2909, 'y1': 1399, 'x2': 3091, 'y2': 1399}",A00000001831_2,A00000001831.jpg,2,Yes,"{""x1"": 840, ""y1"": 343, ""x2"": 942, ""y2"": 340}",102.044108,group_images/A00000001831.jpg,"(3712, 5568, 3)","(1949, 1299, 3)",Chlaenius aestivus,Chlaenius,aestivus,IsaFluck,23248,SERC_010.20180523.CHLAES.01,SERC,2018-05-23,SERC_010,Smithsonian Environmental Research Center NEON,MD,38.89013,-76.56001,GRADIENT,TERRESTRIAL,4.27,26.6,18.9,2.0,6.0,2.0,22.67,D02,0.0,45603.39,61.0,34.6,14.6,resized_images/A00000001831.jpg


In [5]:
print(f'Number of Individual Beetle in the Excel Sheet: {df_bm["beetleID"].nunique()}')

Number of Individual Beetle in the Excel Sheet: 11485


In [6]:
image_dir = "/fs/ess/PAS2136/Rayeed/BeetlePalooza/resized_images/"
image_files = df_bm["pictureID"].unique().tolist()
len(image_files)

577

In [7]:
image_path_list = []
for file in image_files : 
    image_path_list.append(os.path.join(image_dir, file))
image_path_list[:3]

['/fs/ess/PAS2136/Rayeed/BeetlePalooza/resized_images/A00000001831.jpg',
 '/fs/ess/PAS2136/Rayeed/BeetlePalooza/resized_images/A00000003356.jpg',
 '/fs/ess/PAS2136/Rayeed/BeetlePalooza/resized_images/A00000008914.jpg']

In [8]:
# Model and Device Setup
model_id = "IDEA-Research/grounding-dino-base"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Model and Processor
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)

In [9]:
df_bm.head(2)

,beetle_uuid,length_uuid,width_uuid,length_distance_pix,length_distance_cm,length_coord_value,length_coord_value_raw_image,width_distance_pix,width_distance_cm,width_coord_value,width_coord_value_raw_image,beetleID,pictureID,beetle_position,lying_flat,scalebar_coord,pix_per_cm,raw_filepath,raw_image_shape,resized_image_shape,scientificName,genus,species,user_name,workflowID,NEON_sampleID,siteID,collection_date,plotID,siteName,siteState,siteLatitude,siteLongitude,siteType,siteTerrain,evapotranspiration,max_temp,min_temp,precipitation,precipitation_hours,rain_sum,shortwave_radiation,siteDomainCode,snowfall_sum,sunshine_duration,weather_code,wind_gusts_max,wind_speed_max,resized_image_filepath
0,7e443d69-bdcb-4c50-bb56-a1d9142577a6_4e7238e8-aec2-4975-98ad-6e56cc5176c2,7e443d69-bdcb-4c50-bb56-a1d9142577a6,4e7238e8-aec2-4975-98ad-6e56cc5176c2,114.017543,1.117336,"{""x1"": 1047, ""y1"": 230, ""x2"": 1049, ""y2"": 344}","{'x1': 2991, 'y1': 657, 'x2': 2997, 'y2': 982}",65.030762,0.637281,"{""x1"": 1015, ""y1"": 247, ""x2"": 1080, ""y2"": 245}","{'x1': 2900, 'y1': 705, 'x2': 3086, 'y2': 699}",A00000001831_1,A00000001831.jpg,1,Yes,"{""x1"": 840, ""y1"": 343, ""x2"": 942, ""y2"": 340}",102.044108,group_images/A00000001831.jpg,"(3712, 5568, 3)","(1949, 1299, 3)",Chlaenius aestivus,Chlaenius,aestivus,IsaFluck,23248,SERC_010.20180523.CHLAES.01,SERC,2018-05-23,SERC_010,Smithsonian Environmental Research Center NEON,MD,38.89013,-76.56001,GRADIENT,TERRESTRIAL,4.27,26.6,18.9,2.0,6.0,2.0,22.67,D02,0.0,45603.39,61.0,34.6,14.6,resized_images/A00000001831.jpg
1,60aac143-086c-43a6-b506-0a366bba545d_db0890b5-5dfc-408b-b734-4df692166c84,60aac143-086c-43a6-b506-0a366bba545d,db0890b5-5dfc-408b-b734-4df692166c84,109.018347,1.068345,"{""x1"": 1055, ""y1"": 476, ""x2"": 1053, ""y2"": 585}","{'x1': 3014, 'y1': 1359, 'x2': 3009, 'y2': 1671}",64.000000,0.627180,"{""x1"": 1018, ""y1"": 490, ""x2"": 1082, ""y2"": 490}","{'x1': 2909, 'y1': 1399, 'x2': 3091, 'y2': 1399}",A00000001831_2,A00000001831.jpg,2,Yes,"{""x1"": 840, ""y1"": 343, ""x2"": 942, ""y2"": 340}",102.044108,group_images/A00000001831.jpg,"(3712, 5568, 3)","(1949, 1299, 3)",Chlaenius aestivus,Chlaenius,aestivus,IsaFluck,23248,SERC_010.20180523.CHLAES.01,SERC,2018-05-23,SERC_010,Smithsonian Environmental Research Center NEON,MD,38.89013,-76.56001,GRADIENT,TERRESTRIAL,4.27,26.6,18.9,2.0,6.0,2.0,22.67,D02,0.0,45603.39,61.0,34.6,14.6,resized_images/A00000001831.jpg


In [10]:
selected_rows = []

for (beetleID, pictureID), group in df_bm.groupby(['beetleID', 'pictureID']): 
    
    if 'IsaFluck' in group['user_name'].values:
        selected_row = group[group['user_name'] == 'IsaFluck'].iloc[0]
    
    else:
        selected_row = group.iloc[0]
    
    selected_rows.append(selected_row)


df_bm = pd.DataFrame(selected_rows)
df_bm

,beetle_uuid,length_uuid,width_uuid,length_distance_pix,length_distance_cm,length_coord_value,length_coord_value_raw_image,width_distance_pix,width_distance_cm,width_coord_value,width_coord_value_raw_image,beetleID,pictureID,beetle_position,lying_flat,scalebar_coord,pix_per_cm,raw_filepath,raw_image_shape,resized_image_shape,scientificName,genus,species,user_name,workflowID,NEON_sampleID,siteID,collection_date,plotID,siteName,siteState,siteLatitude,siteLongitude,siteType,siteTerrain,evapotranspiration,max_temp,min_temp,precipitation,precipitation_hours,rain_sum,shortwave_radiation,siteDomainCode,snowfall_sum,sunshine_duration,weather_code,wind_gusts_max,wind_speed_max,resized_image_filepath
0,7e443d69-bdcb-4c50-bb56-a1d9142577a6_4e7238e8-aec2-4975-98ad-6e56cc5176c2,7e443d69-bdcb-4c50-bb56-a1d9142577a6,4e7238e8-aec2-4975-98ad-6e56cc5176c2,114.017543,1.117336,"{""x1"": 1047, ""y1"": 230, ""x2"": 1049, ""y2"": 344}","{'x1': 2991, 'y1': 657, 'x2': 2997, 'y2': 982}",65.030762,0.637281,"{""x1"": 1015, ""y1"": 247, ""x2"": 1080, ""y2"": 245}","{'x1': 2900, 'y1': 705, 'x2': 3086, 'y2': 699}",A00000001831_1,A00000001831.jpg,1,Yes,"{""x1"": 840, ""y1"": 343, ""x2"": 942, ""y2"": 340}",102.044108,group_images/A00000001831.jpg,"(3712, 5568, 3)","(1949, 1299, 3)",Chlaenius aestivus,Chlaenius,aestivus,IsaFluck,23248,SERC_010.20180523.CHLAES.01,SERC,2018-05-23,SERC_010,Smithsonian Environmental Research Center NEON,MD,38.89013,-76.56001,GRADIENT,TERRESTRIAL,4.27,26.6,18.9,2.0,6.0,2.0,22.67,D02,0.0,45603.39,61.0,34.6,14.6,resized_images/A00000001831.jpg
9,416b06b7-0d4f-4a03-b56d-0007390d9c8f_68ee2676-de43-43ae-ac10-62e4bfd7d662,416b06b7-0d4f-4a03-b56d-0007390d9c8f,68ee2676-de43-43ae-ac10-62e4bfd7d662,115.212847,1.129049,"{""x1"": 1395, ""y1"": 988, ""x2"": 1402, ""y2"": 1103}","{'x1': 3986, 'y1': 2822, 'x2': 4006, 'y2': 3151}",65.007692,0.637055,"{""x1"": 1364, ""y1"": 1005, ""x2"": 1429, ""y2"": 1004}","{'x1': 3897, 'y1': 2871, 'x2': 4083, 'y2': 2868}",A00000001831_10,A00000001831.jpg,10,Yes,"{""x1"": 840, ""y1"": 343, ""x2"": 942, ""y2"": 340}",102.044108,group_images/A00000001831.jpg,"(3712, 5568, 3)","(1949, 1299, 3)",Chlaenius aestivus,Chlaenius,aestivus,IsaFluck,23248,SERC_010.20180523.CHLAES.01,SERC,2018-05-23,SERC_010,Smithsonian Environmental Research Center NEON,MD,38.89013,-76.56001,GRADIENT,TERRESTRIAL,4.27,26.6,18.9,2.0,6.0,2.0,22.67,D02,0.0,45603.39,61.0,34.6,14.6,resized_images/A00000001831.jpg
10,290e48f1-3ac1-4622-8cc5-559ae444f233_81a42bdf-b5cd-4e98-a5d7-7818eaf56d8f,290e48f1-3ac1-4622-8cc5-559ae444f233,81a42bdf-b5cd-4e98-a5d7-7818eaf56d8f,117.106789,1.147610,"{""x1"": 1430, ""y1"": 228, ""x2"": 1435, ""y2"": 345}","{'x1': 4086, 'y1': 651, 'x2': 4100, 'y2': 985}",66.068147,0.647447,"{""x1"": 1397, ""y1"": 243, ""x2"": 1463, ""y2"": 240}","{'x1': 3992, 'y1': 694, 'x2': 4180, 'y2': 685}",A00000001831_11,A00000001831.jpg,11,Yes,"{""x1"": 840, ""y1"": 343, ""x2"": 942, ""y2"": 340}",102.044108,group_images/A00000001831.jpg,"(3712, 5568, 3)","(1949, 1299, 3)",Chlaenius aestivus,Chlaenius,aestivus,IsaFluck,23248,SERC_010.20180523.CHLAES.01,SERC,2018-05-23,SERC_010,Smithsonian Environmental Research Center NEON,MD,38.89013,-76.56001,GRADIENT,TERRESTRIAL,4.27,26.6,18.9,2.0,6.0,2.0,22.67,D02,0.0,45603.39,61.0,34.6,14.6,resized_images/A00000001831.jpg
11,ae821226-23ca-476b-aabb-ddc4e8d862fe_3e26d0aa-b3e6-40b8-9c22-904f13d2c8e5,ae821226-23ca-476b-aabb-ddc4e8d862fe,3e26d0aa-b3e6-40b8-9c22-904f13d2c8e5,107.004673,1.048612,"{""x1"": 1432, ""y1"": 500, ""x2"": 1431, ""y2"": 607}","{'x1': 4092, 'y1': 1428, 'x2': 4089, 'y2': 1734}",63.000000,0.617380,"{""x1"": 1400, ""y1"": 513, ""x2"": 1463, ""y2"": 513}","{'x1': 4000, 'y1': 1465, 'x2': 4180, 'y2': 1465}",A00000001831_12,A00000001831.jpg,12,Yes,"{""x1"": 840, ""y1"": 343, ""x2"": 942, ""y2"": 340}",102.044108,group_images/A00000001831.jpg,"(3712, 5568, 3)","(1949, 1299, 3)",Chlaenius aestivus,Chlaenius,aestivus,IsaFluck,23248,SERC_010.20180523.CHLAES.01,SERC,2018-05-23,SERC_010,Smithsonian 

In [11]:
def point_inside_box(point, box) :
    
    px, py = point
    bx1, by1, bx2, by2 = box
    
    return bx1 <= px <= bx2 and by1 <= py <= by2


def process_single_image(image_path, df_bm, save_folder, text = "a beetle.", box_threshold=0.2, text_threshold=0.2, padding = 0.1, iou_threshold=0.6): 

    image = Image.open(image_path).convert("RGB")
    img_width, img_height = image.size
    img_area = img_width * img_height
    
    base_name = os.path.basename(image_path)
    image_dir = os.path.join(save_folder, base_name)
    os.makedirs(image_dir, exist_ok=True)
    
    beetles = df_bm[df_bm['pictureID'] == base_name]
    if beetles.empty:
        return
    
    csv_path = os.path.join(image_dir, f"{os.path.splitext(base_name)[0]}.csv")
    
    inputs = processor(images=image, text=text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    
    results = processor.post_process_grounded_object_detection(
        outputs, inputs.input_ids, box_threshold=box_threshold, text_threshold=text_threshold, target_sizes=[image.size[::-1]]
    )
    
    detected_any = False
    
    with open(csv_path, mode='a', newline='') as csv_file:
    
        csv_writer = csv.writer(csv_file)
        
        if os.stat(csv_path).st_size == 0:
            csv_writer.writerow(["pictureID", "beetle_uuid", "x_min", "y_min", "x_max", "y_max", "score"])
        
        for beetle_uuid, beetle_data in beetles.groupby('beetle_uuid'): 
            
            elytra_length = beetle_data['length_coord_value'].dropna().values
            elytra_width = beetle_data['width_coord_value'].dropna().values
            
            elytra_length = eval(elytra_length[0]) if len(elytra_length) > 0 else None
            elytra_width = eval(elytra_width[0]) if len(elytra_width) > 0 else None
            
            elytra_length_line = [(elytra_length['x1'], elytra_length['y1']), (elytra_length['x2'], elytra_length['y2'])] if elytra_length else None
            elytra_width_line = [(elytra_width['x1'], elytra_width['y1']), (elytra_width['x2'], elytra_width['y2'])] if elytra_width else None
            
            best_box = None
            max_area = 0
            best_score = 0
            
            all_boxes = []
            all_scores = []
            
            for result in results:
                num_detections = len(result["boxes"])
                
                for box, score in zip(result["boxes"], result["scores"]):
                    box = [int(coord) for coord in box]
                    bbox_area = (box[2] - box[0]) * (box[3] - box[1])
                    
                    if 5 < num_detections and bbox_area > 0.1 * img_area:
                        continue
                            
                    if 5 <= num_detections < 20 and bbox_area > 0.05 * img_area:
                        continue
                        
                    if 20 <= num_detections < 50 and bbox_area > 0.02 * img_area:
                        continue
                        
                    if 50 <= num_detections < 100 and bbox_area > 0.01 * img_area:
                        continue
                        
                    if 100 <= num_detections < 200 and bbox_area > 0.005 * img_area:
                        continue

                    if 200 <= num_detections and bbox_area > 0.001 * img_area:
                        continue
                    
                    if elytra_length_line and elytra_width_line and \
                       point_inside_box(elytra_length_line[0], box) and point_inside_box(elytra_length_line[1], box) and \
                       point_inside_box(elytra_width_line[0], box) and point_inside_box(elytra_width_line[1], box):
                        
                        all_boxes.append(torch.tensor(box).float())
                        all_scores.append(score)
            

            if len(all_boxes) > 0:
                all_boxes = torch.stack(all_boxes)
                all_scores = torch.tensor(all_scores)
                keep = ops.nms(all_boxes, all_scores, iou_threshold)
                
                for idx in keep:
                    box = all_boxes[idx].int().tolist()
                    score = all_scores[idx].item()
                    
                    bbox_area = (box[2] - box[0]) * (box[3] - box[1])

                    if bbox_area > max_area:
                        max_area = bbox_area
                        best_box = box
                        best_score = score
            
            if best_box :
                
                detected_any = True
                detection_filename = f"{beetle_uuid}.png"
                detection_path = os.path.join(image_dir, detection_filename)
                
                padding_w = int(padding * (best_box[2] - best_box[0]))
                padding_h = int(padding * (best_box[3] - best_box[1]))
                x_min = max(0, best_box[0] - padding_w)
                y_min = max(0, best_box[1] - padding_h)
                x_max = min(img_width, best_box[2] + padding_w)
                y_max = min(img_height, best_box[3] + padding_h)
                
                # cropped_image = image.crop((x_min, y_min, x_max, y_max))
                # cropped_image.save(detection_path, "PNG")

                cropped_image = image.crop((x_min, y_min, x_max, y_max))

                crop_width, crop_height = cropped_image.size
                scale = 512 / max(crop_width, crop_height)
                new_width, new_height = int(crop_width * scale), int(crop_height * scale)
                
                resized_image = cropped_image.resize((new_width, new_height), Image.Resampling.LANCZOS)
                
                imagenet_mean = [int(0.485 * 255), int(0.456 * 255), int(0.406 * 255)]
                padded_image = Image.new("RGB", (512, 512), tuple(imagenet_mean))
                padded_image = Image.new("RGB", (512, 512), (123, 116, 103))
                paste_x = (512 - new_width) // 2
                paste_y = (512 - new_height) // 2
                padded_image.paste(resized_image, (paste_x, paste_y))
                
                padded_image.save(detection_path, "PNG")
                

                csv_writer.writerow([base_name, beetle_uuid, *best_box, round(float(best_score), 4)])
                df_bm.loc[
                    (df_bm["pictureID"] == base_name) & (df_bm["beetle_uuid"] == beetle_uuid),
                    "individual_image_file_path"
                ] = os.path.join("individual_images", base_name, detection_filename)
    
    return detected_any

In [ ]:
outdir = "/fs/ess/PAS2136/Rayeed/BeetlePalooza/Indiv-Res-Images/individual_images/"
os.makedirs(outdir, exist_ok=True)

for col in ["individual_image_file_path", "beetle_coordinates"]:
    if col not in df_bm.columns:
        df_bm[col] = None


detected_images = set()

for image_path in image_path_list:
    if process_single_image(image_path, df_bm, outdir):
        detected_images.add(os.path.basename(image_path))

In [ ]:
df_bm.head(2)

In [ ]:
df_bm["individual_image_file_path"].isna().sum()

In [ ]:
col_order = [
    "beetle_uuid",
    "pictureID",
    "beetleID",
    "beetle_position",
    "scientificName", 
    "genus",
    "species",
    "scalebar_coord", 
    "pix_per_cm",
    "length_uuid", 
    "length_coord_value",
    "length_distance_pix", 
    "length_distance_cm",
    "length_coord_value_raw_image",
    "width_uuid",
    "width_coord_value",
    "width_distance_pix",
    "width_distance_cm",
    "width_coord_value_raw_image", 
    "raw_filepath",
    "raw_image_shape",
    "resized_image_filepath",
    "resized_image_shape",
    "individual_image_file_path",
    "beetle_coordinates",
    "lying_flat", 
    "collection_date", 
    "siteID", 
    "siteName",
    "siteLatitude",
    "siteLongitude",
    "user_name",
    "workflowID",
    "NEON_sampleID",
    "plotID",
    "siteState",
    "siteType",
    "siteTerrain",
    "evapotranspiration",
    "max_temp",
    "min_temp",
    "precipitation",
    "precipitation_hours",
    "rain_sum",
    "shortwave_radiation",
    "siteDomainCode",
    "snowfall_sum",
    "sunshine_duration",
    "weather_code",
    "wind_gusts_max",
    "wind_speed_max", 
]


df_bm = df_bm[col_order]
df_bm.head(3)

In [ ]:
print(f"Numebr of beetles un-Segmented: {df_bm['individual_image_file_path'].isna().sum()}")

In [ ]:
missed = df_bm[df_bm['individual_image_file_path'].isna()]
missed

In [ ]:
# df_bm.to_csv("/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/CSV Update/BeetleMeasurements_Finalized.csv",index=False)